<a href="https://colab.research.google.com/github/Braginsvolitova/Alura_Spark-Class/blob/main/Spark_Regress%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nova seção

In [2]:
!pip install pyspark

In [4]:
from  pyspark.sql import  SparkSession

spark = SparkSession.builder \
  .master('local[*]') \
  .appName ("Regressão com Spark") \
  .getOrCreate()
spark

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
